In [1]:
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
import string
import re
import wandb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers, models

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Constants

In [4]:
BATCH_SIZE = 32
MAX_FEATURES = 10000
SEQ_LENGTH = 250
EMBEDDING_DIM = 300
RNN_UNITS = 64

In [5]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac")

wandb: Currently logged in as: petar-boskovic-ac. Use `wandb login --relogin` to force relogin


Download the artifacts

In [6]:
train_artifact = run.use_artifact('petar-boskovic-ac/fin-sentiment-analysis/stock_data_train_transformed:v0', type='dataset')
test_artifact = run.use_artifact('petar-boskovic-ac/fin-sentiment-analysis/stock_data_test_transformed:v0', type='dataset')
val_artifact = run.use_artifact('petar-boskovic-ac/fin-sentiment-analysis/stock_data_val_transformed:v0', type='dataset')

artifact_dir = train_artifact.download()
artifact_dir = test_artifact.download()
artifact_dir = val_artifact.download()

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [7]:
wandb.finish()

Transform into tfds

In [8]:
def df_to_tfds(dataframe, shuffle=False, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('Sentiment')
    ds = tf.data.Dataset.from_tensor_slices((df['Text'].values, labels.values))
    if shuffle==True:
        ds = ds.shuffle(buffer_size=len(dataframe))
    # ds = ds.batch(batch_size=batch_size)
    return ds

In [9]:
train_df = pd.read_csv('/content/artifacts/stock_data_train_transformed:v0/stock_data_train_transformed.csv')
val_df = pd.read_csv('/content/artifacts/stock_data_val_transformed:v0/stock_data_val_transformed.csv')
test_df = pd.read_csv('/content/artifacts/stock_data_test_transformed:v0/stock_data_test_transformed.csv')

In [10]:
train_ds = df_to_tfds(train_df)
val_ds = df_to_tfds(val_df)
test_ds = df_to_tfds(test_df)

In [11]:
for a,b in train_ds.take(1):
    print(a)
    print(b)

tf.Tensor(b'10 item le made two week shoestring budget writerdirector brad silberling little film shot carson ca feel like entire story improvisedin best sense word silberling good fortune pair veteran actor morgan freeman big project spanish actress paz vega result dialogue two people different vantage manage enhance life morgan freeman play yet part comedy depicted actor work four year scouting location little filler film get back flow thing role market manager dropped seedy market carson encounter among others one scarlet girl argumentative 10 item le checkout line scarlet tired static job also generally angry philandering husband bobby cannavale currently sleeping scarlet lazy coworker anne dudek lack ability get decent job elsewhere two pair shared problem go road trip result character growing presence life story simple story simply told tender bonding freeman paz work well one little film human relationship vulnerable change exchange message well worth viewing dvd featurettes tou

In [12]:
vectorize_layer = TextVectorization(max_tokens=MAX_FEATURES,
                                    output_mode='int',
                                    output_sequence_length=SEQ_LENGTH)

In [13]:
# Adapt the vectorize layer to the training text data
train_text = train_df['Text'].values
vectorize_layer.adapt(train_text)

In [14]:
len(vectorize_layer.get_vocabulary())

10000

In [15]:
vectorize_layer.get_vocabulary()[555]

'message'

In [16]:
def vectorize_text(text, label):
    #text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [17]:
train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

In [18]:
train_ds

<_MapDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [19]:
for a,b in train_ds.take(1):
    print(a)
    print(b)

tf.Tensor(
[ 202 2928  222   28   40  781 7876  289 2463 2849    1   48    3  135
 6275 8358   94    5  332    9    1   47  195  257    1    7 3047 1650
 1845   38 2162 2805  106  802 1830  327 7991 2875  551  285   40   19
  189    1 1710 6493   31 2162 2805   72  153   56  102 2068   38   55
  559   46    1  757   48 4374    3   11   62 2007   27   78 1773 2665
 3081 6136 1773 6275 1482  650  281    4 7137   84    1  202 2928  222
    1  116 7137 1252 4742  179   21 1140 1493    1  482 2725    1 3410
 2638 7137 2707 8238 1661    1  338  878   11  437  179 2909   40 1650
 4848  194   32 1057 1074  551    8 1673 1129   31    9  501    9  232
  457 3890 8587 2805 7991   55   17    4   48    3  250  368 4257  359
 3271  555   17  185  609  171    1 1185 5458  582 1316  451 9981 9018
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0 

In [20]:
train_ds = train_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

## Modeling

###Convolutional Neural Network (CNN)

In [21]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac", name='CONV_1D', config={'model_type': 'CONV 1D',
                                                                                                          'embedding_dim': 300,
                                                                                                          'filters_1': 128,
                                                                                                          'filters_2': 64,
                                                                                                          'kernel_size': 3,
                                                                                                          'padding': 'valid',
                                                                                                          'strides': 1,
                                                                                                          'dropout': 0.2,
                                                                                                          'batch_size': 32,
                                                                                                          'epochs': 15})

In [27]:
cnn = tf.keras.Sequential([tf.keras.layers.Input(shape=(SEQ_LENGTH,)),
                                  tf.keras.layers.Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_DIM),

                                  # Conv1D with BatchNormalization and Dropout
                                  tf.keras.layers.Conv1D(128, 3, activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Conv1D with BatchNormalization and Dropout
                                  tf.keras.layers.Conv1D(64, 3, activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  tf.keras.layers.GlobalMaxPooling1D(),

                                  # Dense with BatchNormalization and Dropout
                                  tf.keras.layers.Dense(64, activation='relu'),
                                  tf.keras.layers.BatchNormalization(),
                                  tf.keras.layers.Dropout(0.2),

                                  # Output Layer
                                  tf.keras.layers.Dense(1, activation='sigmoid')])

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])

cnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 250, 300)            │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 248, 128)            │         115,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 248, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 248, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 246, 64)             │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 246, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 246, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,145,217 (12.00 MB)

 Trainable params: 3,144,705 (12.00 MB)

 Non-trainable params: 512 (2.00 KB)

In [23]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
print("Wandb version:", wandb.__version__)
print("TensorFlow version:", tf.__version__)

Wandb version: 0.17.7
TensorFlow version: 2.17.0


In [24]:
wandb_logger = wandb.keras.WandbMetricsLogger()

In [28]:
history = cnn.fit(train_ds,
                         validation_data=val_ds,
                         epochs = wandb.config.epochs,
                         callbacks=[wandb_logger, early_stopping])

Epoch 1/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 254s 163ms/step - AUC: 0.8059 - accuracy: 0.7301 - loss: 0.5244 - val_AUC: 0.9469 - val_accuracy: 0.8727 - val_loss: 0.3100
Epoch 2/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 208s 160ms/step - AUC: 0.9518 - accuracy: 0.8819 - loss: 0.2802 - val_AUC: 0.9470 - val_accuracy: 0.8548 - val_loss: 0.3333
Epoch 3/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 254s 154ms/step - AUC: 0.9730 - accuracy: 0.9151 - loss: 0.2095 - val_AUC: 0.9424 - val_accuracy: 0.8681 - val_loss: 0.3220
Epoch 4/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 204s 156ms/step - AUC: 0.9865 - accuracy: 0.9432 - loss: 0.1473 - val_AUC: 0.9414 - val_accuracy: 0.8700 - val_loss: 0.3502


In [29]:
cnn.save('cnn.keras')

In [30]:
artifact = wandb.Artifact('cnn1d',
                          type='model',
                          description='CNN based around Conv1D Layer for sentiment analysis',
                          metadata={'epochs': wandb.config.epochs,
                                    'learning_rate': 1e-3,
                                    'batch_size': 32,
                                    'architecture': 'Custom architecture based around Conv1D Layer'})

artifact.add_file('/content/cnn.keras')
wandb.log_artifact(artifact)

wandb.finish()

epoch/AUC,▁▆▇█
epoch/accuracy,▁▅▇█
epoch/epoch,▁▃▆█
epoch/learning_rate,▁▁▁▁
epoch/loss,█▄▂▁
epoch/val_AUC,██▂▁
epoch/val_accuracy,█▁▆▇
epoch/val_loss,▁▅▃█
epoch/AUC,0.9886
epoch/accuracy,0.94843
epoch/epoch,3


####Evaluation

In [31]:
test_ds = test_ds.batch(BATCH_SIZE)

In [32]:
cnn.evaluate(test_ds)

163/163 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - AUC: 0.9480 - accuracy: 0.8748 - loss: 0.3032


[0.31789088249206543, 0.9426514506340027, 0.8646775484085083]

### TextCNN

In [33]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac", name='TextCNN', config={'model_type': 'TextCNN',
                                                                                                          'embedding_dim': 300,
                                                                                                          'filters_1': 128,
                                                                                                          'filters_2': 64,
                                                                                                          'kernel_sizes': (3, 4, 5),
                                                                                                          'padding': 'valid',
                                                                                                          'strides': 1,
                                                                                                          'dropout': 0.2,
                                                                                                          'batch_size': 32,
                                                                                                          'epochs': 15})

In [35]:
cnn_input = tf.keras.layers.Input(shape=(SEQ_LENGTH,))

embedding = tf.keras.layers.Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_DIM)(cnn_input)

# Convolutional layers with different kernel sizes
conv_blocks = []
for kernel_size in [3, 4, 5]:   # Kernel sizes correspond to different n-gram sizes
    conv = tf.keras.layers.Conv1D(filters=128, kernel_size=kernel_size, activation='relu')(embedding)
    conv = tf.keras.layers.GlobalMaxPool1D()(conv)
    conv_blocks.append(conv)

# Concatenate all convolutional outputs
merged = tf.keras.layers.concatenate(conv_blocks, axis=-1)

# Dense layers
dense = tf.keras.layers.Dense(64, activation='relu')(merged)
dropout = tf.keras.layers.Dropout(0.2)(dense)

output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)

# Build model
text_cnn = tf.keras.Model(inputs=cnn_input, outputs=output)

text_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])

text_cnn.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 250)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 250, 300)       │      3,000,000 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_7 (Conv1D)         │ (None, 248, 128)       │        115,328 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_8 (Conv1D)         │ (None, 247, 128)       │        153,728 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_9 (Conv1D)         │ (None, 246, 128)       │        192,128 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_4    │ (None, 128)            │              0 │ conv1d_7[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_5    │ (None, 128)            │              0 │ conv1d_8[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_6    │ (None, 128)            │              0 │ conv1d_9[0][0]         │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 384)            │              0 │ global_max_pooling1d_… │
│ (Concatenate)             │                        │                │ global_max_pooling1d_… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 64)             │         24,640 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 64)             │              0 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 1)              │             65 │ dropout_7[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,485,889 (13.30 MB)

 Trainable params: 3,485,889 (13.30 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
history = text_cnn.fit(train_ds,
                         validation_data=val_ds,
                         epochs = wandb.config.epochs,
                         callbacks=[wandb_logger, early_stopping])

Epoch 1/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 503s 385ms/step - AUC: 0.8659 - accuracy: 0.7859 - loss: 0.4311 - val_AUC: 0.9586 - val_accuracy: 0.8853 - val_loss: 0.2762
Epoch 2/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 532s 409ms/step - AUC: 0.9752 - accuracy: 0.9231 - loss: 0.1975 - val_AUC: 0.9539 - val_accuracy: 0.8698 - val_loss: 0.3581
Epoch 3/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 522s 379ms/step - AUC: 0.9952 - accuracy: 0.9740 - loss: 0.0786 - val_AUC: 0.9462 - val_accuracy: 0.8910 - val_loss: 0.4294
Epoch 4/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 489s 369ms/step - AUC: 0.9984 - accuracy: 0.9828 - loss: 0.0467 - val_AUC: 0.9512 - val_accuracy: 0.8891 - val_loss: 0.3869


In [37]:
text_cnn.save('text_cnn.keras')

In [38]:
artifact = wandb.Artifact('text_cnn',
                          type='model',
                          description='The architecture leverages multiple convolution layers with different kernel sizes to capture n-grams',
                          metadata={'epochs': wandb.config.epochs,
                                    'learning_rate': 1e-3,
                                    'batch_size': 32,
                                    'architecture': 'TextCNN'})

artifact.add_file('/content/text_cnn.keras')
wandb.log_artifact(artifact)

wandb.finish()

epoch/AUC,▁▆██
epoch/accuracy,▁▆██
epoch/epoch,▁▃▆█
epoch/learning_rate,▁▁▁▁
epoch/loss,█▄▂▁
epoch/val_AUC,█▅▁▄
epoch/val_accuracy,▆▁█▇
epoch/val_loss,▁▅█▆
epoch/AUC,0.99826
epoch/accuracy,0.98231
epoch/epoch,3


####Evaluation

In [39]:
text_cnn.evaluate(test_ds)

163/163 ━━━━━━━━━━━━━━━━━━━━ 17s 103ms/step - AUC: 0.9605 - accuracy: 0.8820 - loss: 0.2686


[0.2798115909099579, 0.9570093154907227, 0.8793070316314697]

### Temporal Convolutional Network (TCN)

In [44]:
run = wandb.init(project="fin-sentiment-analysis", entity="petar-boskovic-ac", name='TCN', config={'model_type': 'Temporal Convolutional Network',
                                                                                                          'embedding_dim': 300,
                                                                                                          'filters_1': 128,
                                                                                                          'filters_2': 64,
                                                                                                          'kernel_size': 3,
                                                                                                          'padding': 'valid',
                                                                                                          'strides': 1,
                                                                                                          'dropout': 0.2,
                                                                                                          'batch_size': 32,
                                                                                                          'epochs': 15})

In [40]:
def TCN_Block(filters, kernel_size, dilation_rate):
    return tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='causal', dilation_rate=dilation_rate, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2)
        ])

In [42]:
inputs = tf.keras.Input(shape=(SEQ_LENGTH,))

embedding = tf.keras.layers.Embedding(input_dim=MAX_FEATURES, output_dim=EMBEDDING_DIM)(inputs)

# TCN Blocks
block = TCN_Block(filters=64, kernel_size=3, dilation_rate=1)(embedding)
block = TCN_Block(filters=64, kernel_size=3, dilation_rate=2)(block)
block = TCN_Block(filters=64, kernel_size=3, dilation_rate=2)(block)

pooling = tf.keras.layers.GlobalMaxPooling1D()(block)

# Dense layers
dense = tf.keras.layers.Dense(units=64, activation='relu')(pooling)
dropout = tf.keras.layers.Dropout(0.2)(dense)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)

# Build model
tcn_model = tf.keras.Model(inputs=inputs, outputs=outputs)

tcn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])

tcn_model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)           │ (None, 250)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_5 (Embedding)              │ (None, 250, 300)            │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_5 (Sequential)            │ (None, 250, 64)             │          57,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_6 (Sequential)            │ (None, 250, 64)             │          12,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_7 (Sequential)            │ (None, 250, 64)             │          12,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_8               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,087,361 (11.78 MB)

 Trainable params: 3,086,977 (11.78 MB)

 Non-trainable params: 384 (1.50 KB)

In [45]:
history = tcn_model.fit(train_ds,
                         validation_data=val_ds,
                         epochs = wandb.config.epochs,
                         callbacks=[wandb_logger, early_stopping])

Epoch 1/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 210s 160ms/step - AUC: 0.7362 - accuracy: 0.6737 - loss: 0.6325 - val_AUC: 0.9413 - val_accuracy: 0.8648 - val_loss: 0.3274
Epoch 2/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 188s 144ms/step - AUC: 0.9440 - accuracy: 0.8728 - loss: 0.3024 - val_AUC: 0.9445 - val_accuracy: 0.8708 - val_loss: 0.3072
Epoch 3/15
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 200s 143ms/step - AUC: 0.9652 - accuracy: 0.9028 - loss: 0.2379 - val_AUC: 0.9438 - val_accuracy: 0.8664 - val_loss: 0.3042


In [46]:
tcn_model.save('tcn_model.keras')

In [47]:
artifact = wandb.Artifact('tcn',
                          type='model',
                          description='Handles sequential dependencies while maintaining a causal structure',
                          metadata={'epochs': wandb.config.epochs,
                                    'learning_rate': 1e-3,
                                    'batch_size': 32,
                                    'architecture': 'TCN'})

artifact.add_file('/content/tcn_model.keras')
wandb.log_artifact(artifact)

wandb.finish()

epoch/AUC,▁▇█
epoch/accuracy,▁▇█
epoch/epoch,▁▅█
epoch/learning_rate,▁▁▁
epoch/loss,█▃▁
epoch/val_AUC,▁█▆
epoch/val_accuracy,▁█▃
epoch/val_loss,█▂▁
epoch/AUC,0.96901
epoch/accuracy,0.91098
epoch/epoch,2


####Evaluation

In [48]:
tcn_model.evaluate(test_ds)

163/163 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - AUC: 0.9457 - accuracy: 0.8679 - loss: 0.3160


[0.32689225673675537, 0.940729022026062, 0.8641000986099243]